In [1]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [2]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


E0000 00:00:1735197544.698679   64466 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735197544.701971   64466 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is configured properly.
Num GPUs Available: 1


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [8]:
f.predictions('futu')

FUTU 5m Interval Timestamp: 2024-12-26 03:14:38


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,up,down
kelly_1:2.5,0.208565,0.220455,0.197917,0.054822,0.110859
prob_up,0.187928,0.193771,0.16942,0.625,0.0
prob_static,0.561795,0.536386,0.559921,0.375,0.0
prob_down,0.250277,0.269843,0.270659,0.0,1.0
precision,0.43469,0.443182,0.427083,0.324873,0.364899
recall,0.512626,0.492424,0.517677,0.327366,0.733503
f1,0.470452,0.466507,0.468037,0.326115,0.487352
support,"[391.0, 396.0, 394.0]","[391.0, 396.0, 394.0]","[391.0, 396.0, 394.0]","[391.0, 396.0, 394.0]","[391.0, 396.0, 394.0]"


FUTU 15m Interval Timestamp: 2024-12-26 03:15:08


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,static
kelly_1:2.5,0.256,0.228743,0.295906,0.168317,0.13908
prob_up,0.166593,0.044828,0.269213,0.375,0.0
prob_static,0.784524,0.933938,0.58607,0.5,1.0
prob_down,0.048883,0.021235,0.144716,0.125,0.0
precision,0.468571,0.449102,0.497076,0.405941,0.385057
recall,0.546667,0.5,0.566667,0.546667,0.446667
f1,0.504615,0.473186,0.529595,0.465909,0.41358
support,"[147.0, 150.0, 149.0]","[147.0, 150.0, 149.0]","[147.0, 150.0, 149.0]","[147.0, 150.0, 149.0]","[147.0, 150.0, 149.0]"


FUTU 1h Interval Timestamp: 2024-12-26 03:15:25


...

,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,up,down,static,static
kelly_1:2.5,0.3175,0.08172,0.058407,0.214634,0.068293
prob_up,0.251893,0.415145,0.259625,0.5,0.0
prob_static,0.427718,0.335784,0.353543,0.5,1.0
prob_down,0.320389,0.249072,0.386832,0.0,0.0
precision,0.5125,0.344086,0.327434,0.439024,0.334495
recall,0.427083,0.333333,0.389474,0.5625,1.0
f1,0.465909,0.338624,0.355769,0.493151,0.501305
support,"[96.0, 96.0, 95.0]","[96.0, 96.0, 95.0]","[96.0, 96.0, 95.0]","[96.0, 96.0, 95.0]","[96.0, 96.0, 95.0]"


FUTU 1wk Interval Timestamp: 2024-12-26 03:17:11


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,up,static,static,up
kelly_1:2.5,0.16,0.189474,0.086957,0.024242,0.075
prob_up,0.303446,0.565168,0.33026,0.25,1.0
prob_static,0.431225,0.231062,0.437549,0.375,0.0
prob_down,0.265329,0.20377,0.232191,0.375,0.0
precision,0.4,0.421053,0.347826,0.30303,0.339286
recall,0.421053,0.421053,0.421053,0.526316,1.0
f1,0.410256,0.421053,0.380952,0.384615,0.506667
support,"[19.0, 19.0, 18.0]","[19.0, 19.0, 18.0]","[19.0, 19.0, 18.0]","[19.0, 19.0, 18.0]","[19.0, 19.0, 18.0]"


FUTU 1mo Interval Timestamp: 2024-12-26 03:17:21


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,down
kelly_1:2.5,0.16,0.3,0.16,0.3,0.066667
prob_up,0.00314,0.000002,0.094894,0.25,0.0
prob_static,0.951904,0.999996,0.635821,0.375,0.0
prob_down,0.044956,0.000002,0.269285,0.375,1.0
precision,0.4,0.5,0.4,0.5,0.333333
recall,0.666667,0.666667,0.666667,0.666667,1.0
f1,0.5,0.571429,0.5,0.571429,0.5
support,"[3.0, 3.0, 3.0]","[3.0, 3.0, 3.0]","[3.0, 3.0, 3.0]","[3.0, 3.0, 3.0]","[3.0, 3.0, 3.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')